In [1]:
import MDAnalysis as MDA

Following [this link](http://ipywidgets.readthedocs.io/en/latest/user_install.html) to installl bokeh and ipywidgets and allow notebook extensions for interactivity.

```
pip install ipywidgets bokeh
jupyter nbextension enable --py widgetsnbextension
``` 


In [ ]:
import ipywidgets
from ipywidgets import interact
import os
import numpy as np
import MDAnalysis.analysis.diffusionmap as diffusionmap
import MDAnalysis.analysis.rms as rms
import nglview as ng

In [34]:
print ipywidgets.__version__
print ng.__version__
print np.__version__
print MDA.__version__
print os.getcwd()

5.1.5
0+unknown
1.11.1
0.15.1-dev0
/home/jdetlefs/github/dimension_reduction/diffusionMaps


In [5]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import push_notebook
from bokeh.models import HoverTool, BoxSelectTool


*Be sure to install the adk simulation from [this link](http://becksteinlab.github.io/MDAnalysis-workshop/datadownload.html)*

These are commands you need to input in the terminal in mac or linux to download and unzip:

```
curl -o mdatrj.zip -L 'https://www.dropbox.com/sh/am6y00kac8myihe/AABDiQI28fWnRZueQTT7W2s1a?dl=1'
unzip mdatrj.zip && rm mdatrj.zip
```


In [7]:
u = MDA.Universe('./equilibrium/adk4AKE.psf','./equilibrium/1ake_007-nowater-core-dt240ps.dcd')

In [8]:
dist_step20 = diffusionmap.DistanceMatrix(u, select='backbone', step=20)
%time dist_step20.run()

CPU times: user 1.34 s, sys: 176 ms, total: 1.52 s
Wall time: 1.52 s


In [9]:
dist_step10 = diffusionmap.DistanceMatrix(u, select='backbone', step=10)
%time dist_step10.run()

CPU times: user 5.16 s, sys: 500 ms, total: 5.66 s
Wall time: 5.66 s


In [10]:
dist_step5 = diffusionmap.DistanceMatrix(u, select='backbone', step=5)
%time dist_step5.run()

CPU times: user 20.6 s, sys: 2.18 s, total: 22.8 s
Wall time: 22.8 s


In [11]:
dist_step3 = diffusionmap.DistanceMatrix(u, select='backbone', step=3)
%time dist_step3.run()

CPU times: user 57.6 s, sys: 6.71 s, total: 1min 4s
Wall time: 1min 4s


That took a while for me, let's stop there.

# Using diffusion maps
Now that weve figured out timings,
lets 
+ investigate the distance_matrix
+ pick a reasonable constant epsilon 
+ find the spectrum of reasonable eigenvalues
+ perform an embedding
+ plot the coordinates of the 2d embedding
+ investigate different diffusion spaces

A reasonable value of epsilon corresponds to two elements that are close on a free energy landscape, so lets set epsilon to 1.101 one of our smaller rmsd jumps between frames. Before doing this, let's save the distance matrix, because after determining the epsilon, it will be permanently altered for memory savings.

In [12]:
dmap = diffusionmap.DiffusionMap(dist_step3, epsilon = 5)
dmap.run()

In [13]:
print dist_step3.dist_matrix

[[ 0.          2.17121625  2.1065819  ...,  8.27357091  6.62446111
   6.68169839]
 [ 2.17121625  0.          1.5743044  ...,  7.61275282  6.10934478
   6.15621924]
 [ 2.1065819   1.5743044   0.         ...,  7.49647237  5.9891659
   6.02304391]
 ..., 
 [ 8.27357091  7.61275282  7.49647237 ...,  0.          2.30624856
   2.2490365 ]
 [ 6.62446111  6.10934478  5.9891659  ...,  2.30624856  0.          1.76270925]
 [ 6.68169839  6.15621924  6.02304391 ...,  2.2490365   1.76270925  0.        ]]


In [14]:
# diffusion map the two most dominant eigenvectors
fit = dmap.transform(2, 1)
fit.shape[0]

1396

In [15]:
output_notebook()

Loading BokehJS ...

In [16]:
TOOLS = [BoxSelectTool(), HoverTool()]
#fix range to show effect of scaling better
p = figure(tools=TOOLS, x_range=(-1,1), y_range=(-1,1))
p.title.text = 'coordinates of frames in two dimensional diffusion space'
fit = dmap.transform(2,1)
x = fit[:,0]
y = fit[:,1]
r = p.circle(x,y, fill_alpha=.6)


In [17]:
def update(t=0,vect0=0, vect1=1):
    fit = dmap.transform(time=t)
    x = fit[:,vect0]
    y = fit[:,vect1]
    r.data_source.data['x'] = x
    r.data_source.data['y'] = y
    push_notebook()

In [26]:
show(p)

In [27]:
interact(update, t=(0,10,.01), vect0=(0,10,1), vect1=(0,10,1));

As you can see, most of the points far away in terms of diffusion distance occur only in the beginning! So it looks like right now diffusion mapping only captures the initial closing, but not the opening. (Right now bokeh has an issue with too many hits with the hover tool.)

In [28]:
def diffusion_distance(fit):
    d = np.zeros((fit.shape[0]-1))
    for i in range(fit.shape[0]-1):
        d[i] = (rms.rmsd(fit[i],fit[i+1]))
    return d

In [29]:
dist = diffusion_distance(fit)

In [30]:
TOOLS = [BoxSelectTool(), HoverTool()]
#fix range to show effect of scaling better
p_2 = figure(tools=TOOLS, y_range=(-1,2))
p_2.title.text = 'Diffusion Distance from previous frame'
x_2 = range(dist.shape[0])
y_2 = dist
r1 = p_2.line(x_2[:50], y_2[:50], line_width=2)
r2 = p_2.circle(x_2[:50],y_2[:50], fill_alpha=.6)


In [31]:
def update_dist(dimension=2, t=0, begin=0, end=100):
    fit = dmap.transform(dimension,t)
    dist = diffusion_distance(fit)
    x_2 = range(dist.shape[0])
    y_2 = dist
    r1.data_source.data['x'] = x_2[begin:end]
    r1.data_source.data['y'] = y_2[begin:end]
    r2.data_source.data['x'] = x_2[begin:end]
    r2.data_source.data['y'] = y_2[begin:end]
    push_notebook()

In [32]:
show(p_2)

In [33]:
interact(update_dist, dimension=(1,20,1), t=(0,10,.1), begin = (0,100), end= (10, 1000));

Play with the sliders and there are some interesting insights. When the diffusion space is the first two vectors, we get insight that the large diffusion distance frames occur in the beginning. Increasing the number of vectors in the diffusion space and the another set of frames jumps out between 30 and 50. So maybe 0-20 is an opening and 30-50 is the closing action, the frames need to be inspected with visualization software to understand. 

If you want to investigate the trajectory with nglviewer, restart the 
notebook after running
```
pip install traitlets==4.2.1 ipywidgets==4.1.1 notebook==4.1.0
```
The interactivity will stop working, but the molecular movie can be view to inspect the behavior of the beginning.
```
u = u.select('backbone')
```
```python
import nglview
view = nglview.show_mdanalysis(u)
view
```